## LightGBM-Optunaのサンプルコード

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# LightGBM
import lightgbm as lgb

# Optuna
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
print(lgb.__version__)

3.1.1


In [4]:
print(optuna.__version__)

2.8.0


In [5]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [6]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

In [7]:
def objective(trial):

    lambda_l1 = trial.suggest_loguniform("lambda_l1", 1e-8, 10.0)
    lambda_l2 = trial.suggest_loguniform("lambda_l2", 1e-8, 10.0)
    num_leaves = trial.suggest_int("num_leaves", 2, 256)
    feature_fraction = trial.suggest_uniform("feature_fraction", 0.4, 1.0)
    bagging_fraction = trial.suggest_uniform("bagging_fraction", 0.4, 1.0)
    bagging_freq = trial.suggest_int("bagging_freq", 1, 7)
    min_child_samples = trial.suggest_int("min_child_samples", 5, 100)

    model = lgb.LGBMRegressor(
        lambda_l1=lambda_l1,
        lambda_l2=lambda_l2,
        num_leaves=num_leaves,
        feature_fraction=feature_fraction,
        bagging_fraction=bagging_fraction,
        bagging_freq=bagging_freq,
        min_child_samples=min_child_samples,
    )

    score = cross_val_score(
        model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error"
    )
    mae = score.mean()

    return mae

In [8]:
%%time
# optunaで最適値を見つける
# create_studyメソッドの引数"sampler"にサンプラーと乱数シードを指定
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=500)

[I 2021-07-08 06:21:42,664] A new study created in memory with name: no-name-71018fcc-b9a7-40e8-8898-f841793011e0


[LightGBM] [Warning] lambda_l1 is set=0.08747537025773001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08747537025773001
[LightGBM] [Warning] feature_fraction is set=0.8492823295231671, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8492823295231671
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=1.537331564587801e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.537331564587801e-08
[LightGBM] [Warning] bagging_fraction is set=0.6991042073815543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6991042073815543
[LightGBM] [Warning] lambda_l1 is set=0.08747537025773001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08747537025773001
[LightGBM] [Warning] feature_fraction is set=0.8492823295231671, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8492823295231671
[LightGBM] [Warning] baggin

[I 2021-07-08 06:21:48,768] Trial 0 finished with value: -2.440478380553425 and parameters: {'lambda_l1': 0.08747537025773001, 'lambda_l2': 1.537331564587801e-08, 'num_leaves': 163, 'feature_fraction': 0.8492823295231671, 'bagging_fraction': 0.6991042073815543, 'bagging_freq': 2, 'min_child_samples': 24}. Best is trial 0 with value: -2.440478380553425.


[LightGBM] [Warning] lambda_l1 is set=0.0699481785242808, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0699481785242808
[LightGBM] [Warning] feature_fraction is set=0.8112158910206784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8112158910206784
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=3.32657660618516e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.32657660618516e-07
[LightGBM] [Warning] bagging_fraction is set=0.972036007716962, subsample=1.0 will be ignored. Current value: bagging_fraction=0.972036007716962
[LightGBM] [Warning] lambda_l1 is set=0.0699481785242808, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0699481785242808
[LightGBM] [Warning] feature_fraction is set=0.8112158910206784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8112158910206784
[LightGBM] [Warning] bagging_freq i

[I 2021-07-08 06:21:52,357] Trial 1 finished with value: -2.8110093695251313 and parameters: {'lambda_l1': 0.0699481785242808, 'lambda_l2': 3.32657660618516e-07, 'num_leaves': 24, 'feature_fraction': 0.8112158910206784, 'bagging_fraction': 0.972036007716962, 'bagging_freq': 1, 'min_child_samples': 54}. Best is trial 0 with value: -2.440478380553425.


...

[I 2021-07-08 06:34:22,731] Trial 473 finished with value: -2.135686456709208 and parameters: {'lambda_l1': 0.5381294767423643, 'lambda_l2': 4.8128686653525286, 'num_leaves': 178, 'feature_fraction': 0.8915142938416384, 'bagging_fraction': 0.6718078937611511, 'bagging_freq': 2, 'min_child_samples': 7}. Best is trial 173 with value: -2.0338821786433683.


[LightGBM] [Warning] lambda_l1 is set=0.0017518593063678882, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0017518593063678882
[LightGBM] [Warning] feature_fraction is set=0.5775365455511327, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5775365455511327
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.9096764999933591, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9096764999933591
[LightGBM] [Warning] bagging_fraction is set=0.6191130406445895, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6191130406445895
[LightGBM] [Warning] lambda_l1 is set=0.0017518593063678882, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0017518593063678882
[LightGBM] [Warning] feature_fraction is set=0.5775365455511327, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5775365455511327
[LightGBM] [Warning] bagg

[I 2021-07-08 06:34:23,065] Trial 474 finished with value: -2.105972434568282 and parameters: {'lambda_l1': 0.0017518593063678882, 'lambda_l2': 0.9096764999933591, 'num_leaves': 129, 'feature_fraction': 0.5775365455511327, 'bagging_fraction': 0.6191130406445895, 'bagging_freq': 2, 'min_child_samples': 5}. Best is trial 173 with value: -2.0338821786433683.


[LightGBM] [Warning] lambda_l1 is set=0.0017518593063678882, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0017518593063678882
[LightGBM] [Warning] feature_fraction is set=0.5775365455511327, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5775365455511327
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.9096764999933591, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9096764999933591
[LightGBM] [Warning] bagging_fraction is set=0.6191130406445895, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6191130406445895
[LightGBM] [Warning] lambda_l1 is set=0.0012166777997067815, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0012166777997067815
[LightGBM] [Warning] feature_fraction is set=0.9302960441153634, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9302960441153634
[LightGBM] [Warning] bagg

KeyboardInterrupt: 

In [9]:
# チューニングしたハイパーパラメータ
study.best_params

{'lambda_l1': 1.0524650666993525,
 'lambda_l2': 0.9207198775866577,
 'num_leaves': 218,
 'feature_fraction': 0.9303002990145904,
 'bagging_fraction': 0.652299050498286,
 'bagging_freq': 2,
 'min_child_samples': 7}

In [10]:
# チューニングしたハイパーパラメーターをフィット
optimised_model = lgb.LGBMRegressor(
    lambda_l1=study.best_params["lambda_l1"],
    lambda_l2=study.best_params["lambda_l2"],
    num_leaves=study.best_params["num_leaves"],
    feature_fraction=study.best_params["feature_fraction"],
    bagging_fraction=study.best_params["bagging_fraction"],
    bagging_freq=study.best_params["bagging_freq"],
    min_child_samples=study.best_params["min_child_samples"],
)

optimised_model.fit(x_train, y_train)

# LightGBM推論
y_pred = optimised_model.predict(x_test)

In [11]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [12]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.835922  2.643337  17.159429  4.142394
